In [381]:
import os, glob
import pandas as pd

# Transform data only run once

In [382]:
# PATH = "/Users/apple/Desktop/Development/freelance/upwork/brad-miles/Landmarks-Detection/Dataset/Right"
# NEW_PATH = "/Users/apple/Desktop/Development/freelance/upwork/brad-miles/Landmarks-Detection/csvs/"
# files = glob.glob(PATH + '/*/Landmarks_femur.csv', recursive=True)
# text = "landmark,x, y, z\n"
# for file in files:
#     with open(file) as f:
#         data = f.read()
#     new_data = text + data
#     splited = file.rsplit('/', 2)
#     path_to_store = NEW_PATH + "{} {}".format(splited[1], splited[2])
#     with open(path_to_store, "w+") as fn:
#         fn.write(new_data)


# Remove femCMPImportREF

## Only run once

In [383]:
# PATH = "/Users/apple/Desktop/Development/freelance/upwork/brad-miles/Landmarks-Detection/csvs/"
# files = glob.glob(PATH + '/*.csv', recursive=True)
# for file in files:
#     with open(file) as f:
#         data = f.readlines()
#     if "femCMPImportREF" in data[-1]:
#         new_data = data[:len(data)-1]
#     else:
#         new_data = data
#     new_data = "".join(new_data)
#     splited = file.rsplit('/', 2)
#     path_to_store = file
#     with open(path_to_store, "w+") as fn:
#         fn.write(new_data)


# Data Preprocessing

In [384]:
PATH = "/Users/apple/Desktop/Development/freelance/upwork/brad-miles/Landmarks-Detection/csvs/"
files = glob.glob(PATH + '/*.csv', recursive=True)

In [385]:
#defining known and unknown landmarks
landmarks = [
    'PCLOrigin',
    'femoralCenter',
    'greaterTrochanter',
    'hipCenter',
    'lateralCondyle',
    'lateralEpicondyle',
    'lateralPosteriorCondyle',
    'medialCondyle',
    'medialEpicondyle',
    'medialPosteriorCondyle',
    'medialSulcus',
    'midfemurCenter',
    'whitesideReference'
]
#    'femoralCenter',
known_landmarks = [
    'greaterTrochanter',
    'hipCenter',
    'lateralCondyle',
    'lateralPosteriorCondyle',
    'medialCondyle',
    'medialPosteriorCondyle',
    'midfemurCenter'
]

unknown_landmarks = [
    'PCLOrigin',
    'lateralCondyle',
    'medialCondyle',
    'medialSulcus',
    'femoralCenter',
    'whitesideReference'
]


In [386]:
# Loading train dataset
df = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file).transpose()
    temp = []
    for item in df1.iloc[0]:
        temp.append(item)
    df1.columns = temp
    df = df.append(df1[1:4])

In [387]:
df.head()

,hipCenter,greaterTrochanter,femoralCenter,whitesideReference,lateralCondyle,medialCondyle,lateralPosteriorCondyle,medialPosteriorCondyle,midfemurCenter,PCLOrigin,lateralEpicondyle,medialSulcus,medialEpicondyle
x,-101.356,-151.015,-97.319,-101.918,-118.063,-69.047,-112.612,-58.7281,-120.973,-80.9237,-132.661,-51.0437,-52.0821
y,-49.422,-22.8869,-17.3471,-41.1628,-8.5737,-17.3388,22.567,12.1943,-39.4207,-1.4492,0.58585,-14.826,-21.6239
z,846.434,851.429,429.888,443.705,424.892,422.386,445.66,446.113,619.25,452.959,448.878,446.386,449.6
x,-89.26,-129.98,-109.27,-112.14,-131.56,-85.94,-123.89,-80.18,-117.21,-96.01,-142.79,-69.91,-69.19
y,6.19,32.98,-29.51,-47.73,-17.07,-21.99,8.4,3.57,-16.12,-11.63,-5.86,-21.02,-26.75


In [388]:
# spliting trin dataset
df_x = df[known_landmarks]
df_y = df[unknown_landmarks]

In [389]:
df_x.head()

,greaterTrochanter,hipCenter,lateralCondyle,lateralPosteriorCondyle,medialCondyle,medialPosteriorCondyle,midfemurCenter
x,-151.015,-101.356,-118.063,-112.612,-69.047,-58.7281,-120.973
y,-22.8869,-49.422,-8.5737,22.567,-17.3388,12.1943,-39.4207
z,851.429,846.434,424.892,445.66,422.386,446.113,619.25
x,-129.98,-89.26,-131.56,-123.89,-85.94,-80.18,-117.21
y,32.98,6.19,-17.07,8.4,-21.99,3.57,-16.12


In [390]:
df_y.head()

,PCLOrigin,lateralCondyle,medialCondyle,medialSulcus,femoralCenter,whitesideReference
x,-80.9237,-118.063,-69.047,-51.0437,-97.319,-101.918
y,-1.4492,-8.5737,-17.3388,-14.826,-17.3471,-41.1628
z,452.959,424.892,422.386,446.386,429.888,443.705
x,-96.01,-131.56,-85.94,-69.91,-109.27,-112.14
y,-11.63,-17.07,-21.99,-21.02,-29.51,-47.73


In [391]:
# Transforming dataframe so that each patient is represented as 1 example to model

new_known_landmarks = []
for item in df_x.columns:
    new_known_landmarks.append(item + "_x")
    new_known_landmarks.append(item+"_y")
    new_known_landmarks.append(item+"_z")
    
new_df = []
for i in range(0, len(df_x), 3):
    row = [] 
    temp = df_x.iloc(0)
    for item in known_landmarks:
        row.append(temp[i][item])
        row.append(temp[i+1][item])
        row.append(temp[i+2][item])
    new_df.append(row)
new_df_x = pd.DataFrame(new_df)
new_df_x.columns = new_known_landmarks

In [392]:
new_df_x.head()

,greaterTrochanter_x,greaterTrochanter_y,greaterTrochanter_z,hipCenter_x,hipCenter_y,hipCenter_z,lateralCondyle_x,lateralCondyle_y,lateralCondyle_z,lateralPosteriorCondyle_x,...,lateralPosteriorCondyle_z,medialCondyle_x,medialCondyle_y,medialCondyle_z,medialPosteriorCondyle_x,medialPosteriorCondyle_y,medialPosteriorCondyle_z,midfemurCenter_x,midfemurCenter_y,midfemurCenter_z
0,-151.0151,-22.8869,851.4287,-101.3562,-49.4220,846.4340,-118.06260,-8.57370,424.89170,-112.61225,...,445.6599,-69.04705,-17.33875,422.38560,-58.72815,12.19430,446.11305,-120.9726,-39.4207,619.25000
1,-129.9800,32.9800,837.5700,-89.2600,6.1900,827.4100,-131.56000,-17.07000,419.70000,-123.89000,...,435.7000,-85.94000,-21.99000,410.63000,-80.18000,3.57000,430.42000,-117.2100,-16.1200,628.12000
2,-134.6207,0.0528,890.0362,-87.1589,-19.0980,885.5397,-107.52645,-6.75915,439.39165,-104.57975,...,459.0545,-56.43075,-5.40480,433.13375,-55.96670,21.00665,454.48445,-98.7390,-25.1139,670.61435
3,-130.3166,1.9774,812.3120,-78.6474,-16.0195,809.2155,-109.12725,-19.43160,412.62605,-109.25795,...,432.3340,-59.24775,-18.84650,408.88080,-59.36025,10.25410,433.28645,-104.3989,-28.7140,606.36135
4,-137.7000,5.1900,851.6400,-87.3900,-0.9200,846.8800,-115.95000,12.55000,398.12000,-117.01000,...,423.6600,-59.70000,15.92000,396.77000,-62.42000,47.37000,421.51000,-107.8300,-7.6700,635.62000


In [393]:
# Transforming dataframe so that each patient is represented as 1 example to model

new_unknown_landmarks = []
for item in unknown_landmarks:
    new_unknown_landmarks.append(item + "_x")
    new_unknown_landmarks.append(item+"_y")
    new_unknown_landmarks.append(item+"_z")
    
new_df = []
for i in range(0, len(df_y), 3):
    row = [] 
    temp = df_y.iloc(0)
    for item in unknown_landmarks:
        row.append(temp[i][item])
        row.append(temp[i+1][item])
        row.append(temp[i+2][item])
    new_df.append(row)
    
new_df_y = pd.DataFrame(new_df)
new_df_y.columns = new_unknown_landmarks

In [394]:
new_df_y["PCLOrigin_x"].head()

0   -80.92375
1   -96.01000
2   -69.62610
3   -76.16965
4   -77.49000
Name: PCLOrigin_x, dtype: float64

In [395]:
new_df_x.head()

,greaterTrochanter_x,greaterTrochanter_y,greaterTrochanter_z,hipCenter_x,hipCenter_y,hipCenter_z,lateralCondyle_x,lateralCondyle_y,lateralCondyle_z,lateralPosteriorCondyle_x,...,lateralPosteriorCondyle_z,medialCondyle_x,medialCondyle_y,medialCondyle_z,medialPosteriorCondyle_x,medialPosteriorCondyle_y,medialPosteriorCondyle_z,midfemurCenter_x,midfemurCenter_y,midfemurCenter_z
0,-151.0151,-22.8869,851.4287,-101.3562,-49.4220,846.4340,-118.06260,-8.57370,424.89170,-112.61225,...,445.6599,-69.04705,-17.33875,422.38560,-58.72815,12.19430,446.11305,-120.9726,-39.4207,619.25000
1,-129.9800,32.9800,837.5700,-89.2600,6.1900,827.4100,-131.56000,-17.07000,419.70000,-123.89000,...,435.7000,-85.94000,-21.99000,410.63000,-80.18000,3.57000,430.42000,-117.2100,-16.1200,628.12000
2,-134.6207,0.0528,890.0362,-87.1589,-19.0980,885.5397,-107.52645,-6.75915,439.39165,-104.57975,...,459.0545,-56.43075,-5.40480,433.13375,-55.96670,21.00665,454.48445,-98.7390,-25.1139,670.61435
3,-130.3166,1.9774,812.3120,-78.6474,-16.0195,809.2155,-109.12725,-19.43160,412.62605,-109.25795,...,432.3340,-59.24775,-18.84650,408.88080,-59.36025,10.25410,433.28645,-104.3989,-28.7140,606.36135
4,-137.7000,5.1900,851.6400,-87.3900,-0.9200,846.8800,-115.95000,12.55000,398.12000,-117.01000,...,423.6600,-59.70000,15.92000,396.77000,-62.42000,47.37000,421.51000,-107.8300,-7.6700,635.62000


In [396]:
def nans(df): return df[df.isnull().any(axis=1)]
nans(new_df_x)

,greaterTrochanter_x,greaterTrochanter_y,greaterTrochanter_z,hipCenter_x,hipCenter_y,hipCenter_z,lateralCondyle_x,lateralCondyle_y,lateralCondyle_z,lateralPosteriorCondyle_x,...,lateralPosteriorCondyle_z,medialCondyle_x,medialCondyle_y,medialCondyle_z,medialPosteriorCondyle_x,medialPosteriorCondyle_y,medialPosteriorCondyle_z,midfemurCenter_x,midfemurCenter_y,midfemurCenter_z


# Model Training

In [397]:
from sklearn import linear_model

## PCLOrigin training

In [398]:
lm_x_po = linear_model.LinearRegression()
model_x = lm_x_po.fit(new_df_x,new_df_y["PCLOrigin_x"])
lm_y_po = linear_model.LinearRegression()
model_y = lm_y_po.fit(new_df_x,new_df_y["PCLOrigin_y"])
lm_z_po = linear_model.LinearRegression()
model_z = lm_z_po.fit(new_df_x,new_df_y["PCLOrigin_z"])

## lateralCondyle training

In [399]:
lm_x_lc = linear_model.LinearRegression()
model_x = lm_x_lc.fit(new_df_x,new_df_y["lateralCondyle_x"])
lm_y_lc = linear_model.LinearRegression()
model_y = lm_y_lc.fit(new_df_x,new_df_y["lateralCondyle_y"])
lm_z_lc = linear_model.LinearRegression()
model_z = lm_z_lc.fit(new_df_x,new_df_y["lateralCondyle_z"])

## medialCondyle training

In [400]:
lm_x_mc = linear_model.LinearRegression()
model_x = lm_x_mc.fit(new_df_x,new_df_y["medialCondyle_x"])
lm_y_mc = linear_model.LinearRegression()
model_y = lm_y_mc.fit(new_df_x,new_df_y["medialCondyle_y"])
lm_z_mc = linear_model.LinearRegression()
model_z = lm_z_mc.fit(new_df_x,new_df_y["medialCondyle_z"])

## medialSulcus training

In [401]:
lm_x_ms = linear_model.LinearRegression()
model_x = lm_x_ms.fit(new_df_x,new_df_y["medialSulcus_x"])
lm_y_ms = linear_model.LinearRegression()
model_y = lm_y_ms.fit(new_df_x,new_df_y["medialSulcus_y"])
lm_z_ms = linear_model.LinearRegression()
model_z = lm_z_ms.fit(new_df_x,new_df_y["medialSulcus_z"])

## whitesideReference training

In [402]:
lm_x_wr = linear_model.LinearRegression()
model_x = lm_x_wr.fit(new_df_x,new_df_y["whitesideReference_x"])
lm_y_wr = linear_model.LinearRegression()
model_y = lm_y_wr.fit(new_df_x,new_df_y["whitesideReference_y"])
lm_z_wr = linear_model.LinearRegression()
model_z = lm_z_wr.fit(new_df_x,new_df_y["whitesideReference_z"])

## femoralCenter training

In [403]:
lm_x_fc = linear_model.LinearRegression()
model_x = lm_x_fc.fit(new_df_x,new_df_y["femoralCenter_x"])
lm_y_fc = linear_model.LinearRegression()
model_y = lm_y_fc.fit(new_df_x,new_df_y["femoralCenter_y"])
lm_z_fc = linear_model.LinearRegression()
model_z = lm_z_fc.fit(new_df_x,new_df_y["femoralCenter_z"])

# Prediction testing


In [404]:
PATH = "/Users/apple/Desktop/Development/freelance/upwork/brad-miles/Landmarks-Detection/csvs-test/"
files = glob.glob(PATH + '/*.csv', recursive=True)

In [405]:
# Loading test dataset
df = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file).transpose()
    temp = []
    for item in df1.iloc[0]:
        temp.append(item)
    df1.columns = temp
    df = df.append(df1[1:4])

In [406]:
# spliting test dataset
df_x = df[known_landmarks]
df_y = df[unknown_landmarks]

In [407]:
# transforming test dataset for prediction
new_known_landmarks = []
for item in df_x.columns:
    new_known_landmarks.append(item + "_x")
    new_known_landmarks.append(item+"_y")
    new_known_landmarks.append(item+"_z")
    
new_df = []
for i in range(0, len(df_x), 3):
    row = [] 
    temp = df_x.iloc(0)
    for item in known_landmarks:
        row.append(temp[i][item])
        row.append(temp[i+1][item])
        row.append(temp[i+2][item])
    new_df.append(row)
new_df_x = pd.DataFrame(new_df)
new_df_x.columns = new_known_landmarks

In [408]:
# transforming test dataset for prediction

new_unknown_landmarks = []
for item in unknown_landmarks:
    new_unknown_landmarks.append(item + "_x")
    new_unknown_landmarks.append(item+"_y")
    new_unknown_landmarks.append(item+"_z")
    
new_df = []
for i in range(0, len(df_y), 3):
    row = [] 
    temp = df_y.iloc(0)
    for item in unknown_landmarks:
        row.append(temp[i][item])
        row.append(temp[i+1][item])
        row.append(temp[i+2][item])
    new_df.append(row)
    
new_df_y = pd.DataFrame(new_df)
new_df_y.columns = new_unknown_landmarks

## PCL_orgin_test

In [409]:
#pedict PCL_origin 
predictions_x = lm_x_po.predict(new_df_x)
predictions_y = lm_y_po.predict(new_df_x)
predictions_z = lm_z_po.predict(new_df_x)
PCL_origin_predictions = []
for i, x in enumerate(predictions_z):
    PCL_origin_predictions.append([predictions_x[i], predictions_y[i], x])

In [410]:
PCL_origin_df = pd.DataFrame(PCL_origin_predictions)
PCL_origin_df.columns= ["PCLOrigin_x", "PCLOrigin_y", "PCLOrigin_z"]
print("\nPredicted Values!!!")
PCL_origin_df


Predicted Values!!!


,PCLOrigin_x,PCLOrigin_y,PCLOrigin_z
0,-108.530194,-12.740676,380.072502
1,-81.759348,12.596128,557.194762
2,-65.524508,22.291974,475.014807
3,-87.307729,-22.672814,434.429999


In [411]:
print("\nActual Values!!!")
new_df_y[["PCLOrigin_x", "PCLOrigin_y", "PCLOrigin_z"]]


Actual Values!!!


,PCLOrigin_x,PCLOrigin_y,PCLOrigin_z
0,-107.28910,-12.63850,380.89080
1,-82.68090,13.10415,560.22745
2,-68.13000,20.66000,472.98000
3,-89.59015,-23.54265,437.80365


## lateralCondyle_test

In [412]:
#pedict lateralCondyle 
predictions_x = lm_x_lc.predict(new_df_x)
predictions_y = lm_y_lc.predict(new_df_x)
predictions_z = lm_z_lc.predict(new_df_x)
lateralCondyle_predictions = []
for i, x in enumerate(predictions_z):
    lateralCondyle_predictions.append([predictions_x[i], predictions_y[i], x])

In [413]:
lateralCondyle_df = pd.DataFrame(lateralCondyle_predictions)
lateralCondyle_df.columns= ["lateralCondyle_x", "lateralCondyle_y", "lateralCondyle_z"]
print("\nPredicted Values!!!")
lateralCondyle_df


Predicted Values!!!


,lateralCondyle_x,lateralCondyle_y,lateralCondyle_z
0,-139.60375,-19.1474,358.1340
1,-113.56680,-10.6905,527.9941
2,-99.05000,1.3500,445.6500
3,-123.12315,-40.3478,405.6334


In [414]:
print("\nActual Values!!!")
new_df_y[["lateralCondyle_x", "lateralCondyle_y", "lateralCondyle_z"]]


Actual Values!!!


,lateralCondyle_x,lateralCondyle_y,lateralCondyle_z
0,-139.60375,-19.1474,358.1340
1,-113.56680,-10.6905,527.9941
2,-99.05000,1.3500,445.6500
3,-123.12315,-40.3478,405.6334


## medialCondyle_test

In [415]:
#predict medialCondyle
predictions_x = lm_x_mc.predict(new_df_x)
predictions_y = lm_y_mc.predict(new_df_x)
predictions_z = lm_z_mc.predict(new_df_x)
medialCondyle_predictions = []
for i, x in enumerate(predictions_z):
    medialCondyle_predictions.append([predictions_x[i], predictions_y[i], x])

In [416]:
medialCondyle_df = pd.DataFrame(medialCondyle_predictions)
medialCondyle_df.columns= ["medialCondyle_x", "medialCondyle_y", "medialCondyle_z"]
print("\nPredicted Values!!!")
medialCondyle_df


Predicted Values!!!


,medialCondyle_x,medialCondyle_y,medialCondyle_z
0,-96.2454,-23.9203,352.30405
1,-63.5973,3.8674,525.63820
2,-47.3700,16.9800,444.39000
3,-68.3172,-34.2709,403.40675


In [417]:
print("\nActual Values!!!")
new_df_y[["medialCondyle_x", "medialCondyle_y", "medialCondyle_z"]]


Actual Values!!!


,medialCondyle_x,medialCondyle_y,medialCondyle_z
0,-96.2454,-23.9203,352.30405
1,-63.5973,3.8674,525.63820
2,-47.3700,16.9800,444.39000
3,-68.3172,-34.2709,403.40675


## medialSulcus_test

In [418]:
#predict medialSulcus

predictions_x = lm_x_ms.predict(new_df_x)
predictions_y = lm_y_ms.predict(new_df_x)
predictions_z = lm_z_ms.predict(new_df_x)
medialSulcus_predictions = []
for i, x in enumerate(predictions_z):
    medialSulcus_predictions.append([predictions_x[i], predictions_y[i], x])

In [419]:
medialSulcus_df = pd.DataFrame(medialSulcus_predictions)
medialSulcus_df.columns= ["medialSulcus_x", "medialSulcus_y", "medialSulcus_z"]
print("\nPredicted Values!!!")
medialSulcus_df


Predicted Values!!!


,medialSulcus_x,medialSulcus_y,medialSulcus_z
0,-81.024897,-20.192616,373.432367
1,-46.974476,13.030217,554.249883
2,-32.387447,26.680403,471.792581
3,-52.474320,-26.918729,430.894108


In [420]:
print("\nActual Values!!!")
new_df_y[["medialSulcus_x", "medialSulcus_y", "medialSulcus_z"]]


Actual Values!!!


,medialSulcus_x,medialSulcus_y,medialSulcus_z
0,-80.71720,-24.18990,374.18985
1,-46.41230,13.80890,548.12240
2,-32.55000,24.91000,473.45000
3,-51.20975,-26.13155,430.85770


## whitesideReference_test

In [421]:
#predict whitesideReference
predictions_x = lm_x_wr.predict(new_df_x)
predictions_y = lm_y_wr.predict(new_df_x)
predictions_z = lm_z_wr.predict(new_df_x)
whitesideReference_predictions = []
for i, x in enumerate(predictions_z):
    whitesideReference_predictions.append([predictions_x[i], predictions_y[i], x])

In [422]:
whitesideReference_df = pd.DataFrame(whitesideReference_predictions)
whitesideReference_df.columns= ["whitesideReference_x", "whitesideReference_y", "whitesideReference_z"]
print("\nPredicted Values!!!")
whitesideReference_df


Predicted Values!!!


,whitesideReference_x,whitesideReference_y,whitesideReference_z
0,-120.301375,-47.274350,373.688711
1,-83.557695,-33.140173,552.908726
2,-65.773584,-21.250107,467.845886
3,-95.456071,-66.689429,429.291262


In [423]:
print("\nActual Values!!!")
new_df_y[["whitesideReference_x", "whitesideReference_y", "whitesideReference_z"]]


Actual Values!!!


,whitesideReference_x,whitesideReference_y,whitesideReference_z
0,-120.5439,-47.79950,374.66860
1,-83.5782,-28.79595,547.52225
2,-65.2600,-20.94000,463.60000
3,-97.4795,-71.40845,429.12665


## femoralCenter_test

In [424]:
#predict whitesideReference
predictions_x = lm_x_fc.predict(new_df_x)
predictions_y = lm_y_fc.predict(new_df_x)
predictions_z = lm_z_fc.predict(new_df_x)
femoralCenter_predictions = []
for i, x in enumerate(predictions_z):
    femoralCenter_predictions.append([predictions_x[i], predictions_y[i], x])

In [425]:
femoralCenter_df = pd.DataFrame(femoralCenter_predictions)
femoralCenter_df.columns= ["femoralCenter_x", "femoralCenter_y", "femoralCenter_z"]
print("\nPredicted Values!!!")
femoralCenter_df


Predicted Values!!!


,femoralCenter_x,femoralCenter_y,femoralCenter_z
0,-120.313166,-31.010080,361.170159
1,-87.120033,-11.731893,535.074409
2,-70.207432,-1.998096,453.872713
3,-94.776860,-45.289362,412.182223


In [426]:
print("\nActual Values!!!")
new_df_y[["femoralCenter_x", "femoralCenter_y", "femoralCenter_z"]]


Actual Values!!!


,femoralCenter_x,femoralCenter_y,femoralCenter_z
0,-121.18030,-29.57105,361.87770
1,-89.33175,-10.72330,534.83485
2,-70.89000,-1.94000,451.89000
3,-96.30025,-47.43390,410.64670
